In [1]:
print("ok")

ok


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


c:\Users\dell\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
!pip install pypdf


In [4]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader

import os

def load_pdf(data):
    if not os.path.isdir(data):
        raise FileNotFoundError(f"Directory not found: '{data}'")
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

extrated_data = load_pdf("data/")




In [5]:
import os
print(os.getcwd())


d:\ai\End-to-end-Medical-Chatbot-using-Llama2-main\End-to-end-Medical-Chatbot-using-Llama2-main


In [6]:
#text chunks

def text_split(extrated_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extrated_data)

    return text_chunks

In [7]:
text_chunks=text_split(extrated_data)
print("length of my chunks",len(text_chunks))

length of my chunks 16191


In [8]:
#embedding model
def download_hugging_face_embedding():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings=download_hugging_face_embedding()

C:\Users\dell\AppData\Local\Temp\ipykernel_36784\3682888065.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
embeddings


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
pcsk_6UQ53m_BTsLJJ9vCx8YTWBjwBNqRgmFbLdtn3LSkVthjuhczndo7D1wGBFyX284gLNVm27

In [14]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain_huggingface import HuggingFaceEmbeddings

# Set Pinecone API Key
PINECONE_API_KEY ="pcsk_6UQ53m_BTsLJJ9vCx8YTWBjwBNqRgmFbLdtn3LSkVthjuhczndo7D1wGBFyX284gLNVm27"
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY  # Explicitly setting the key

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define index name
index_name = "medical-chat"

# Check if the index already exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Ensure it matches your embedding model
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Connect to the existing index
index = pc.Index(index_name)

# Load HuggingFace embeddings (use the updated import)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Ensure text_chunks is a list of raw strings
if not isinstance(text_chunks, list) or not all(isinstance(t, str) for t in text_chunks):
    # If text_chunks is a list of objects with 'page_content' attributes
    if hasattr(text_chunks[0], 'page_content'):
        text_chunks = [t.page_content for t in text_chunks]  # Extract 'page_content'
    else:
        raise ValueError("text_chunks must be a list of strings or have 'page_content' attribute.")

# Convert text chunks into embeddings and store them in Pinecone
docsearch = LangchainPinecone.from_texts(
    texts=text_chunks,  # Pass the list of text chunks (now guaranteed to be strings)
    embedding=embeddings,
    index_name=index_name
)

print("Documents indexed successfully!")


Documents indexed successfully!


In [13]:
from langchain.vectorstores import Pinecone
from langchain_huggingface import HuggingFaceEmbeddings
docsearch=Pinecone.from_existing_index(index_name,embeddings)
query="what are allergies"
docs=docsearch.similarity_search(query,k=3)
print("result",docs)

result [Document(page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'), Document(page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'), Document(page_content='Purpose

In [14]:
from langchain.prompts import PromptTemplate

# Define the prompt template
promptTemplate = """
You are a helpful assistant. 
Context: {context}
Question: {question}
"""

# Create the PromptTemplate object with the defined template
PROMPT = PromptTemplate(template=promptTemplate, input_variables=["context", "question"])

# Define chain_type_kwargs (if you're using a chain)
chain_type_kwargs = {"prompt": PROMPT}


In [15]:
llm=CTransformers(model="model\llama-2-7b-chat.ggmlv3.q4_0.bin",model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [16]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k":2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [17]:
while True:
    user_in=input(f"input prompt:")
    result=qa({"query":user_in})
    print("response:",result["result"])

C:\Users\dell\AppData\Local\Temp\ipykernel_39668\227681466.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result=qa({"query":user_in})


response: Answer: Sorry to hear that you're struggling with depression. It's important to know that depression is a common and treatable mental health condition. Here are some possible symptoms of depression that you might be experiencing:

* Persistent feelings of sadness, hopelessness, or emptiness
* Loss of interest in activities that were once enjoyed
* Changes in appetite or sleep patterns
* Fatigue or loss of energy
* Difficulty concentrating or making decisions
* Thoughts of death or suicide

It's important to remember that these symptoms can vary from person to person, and not everyone will experience all of them. If you think you might be experiencing depression, it's important to reach out to a mental health professional for an evaluation and treatment. They can help you develop coping strategies, provide support, and prescribe medication if necessary.

Remember, depression is not something to be ashamed of, and seeking help is a sign of strength.


Number of tokens (513) exceeded maximum context length (512).
Number of tokens (514) exceeded maximum context length (512).
Number of tokens (515) exceeded maximum context length (512).
Number of tokens (516) exceeded maximum context length (512).
Number of tokens (517) exceeded maximum context length (512).
Number of tokens (518) exceeded maximum context length (512).
Number of tokens (519) exceeded maximum context length (512).
Number of tokens (520) exceeded maximum context length (512).
Number of tokens (521) exceeded maximum context length (512).
Number of tokens (522) exceeded maximum context length (512).
Number of tokens (523) exceeded maximum context length (512).
Number of tokens (524) exceeded maximum context length (512).
Number of tokens (525) exceeded maximum context length (512).
Number of tokens (526) exceeded maximum context length (512).
Number of tokens (527) exceeded maximum context length (512).
Number of tokens (528) exceeded maximum context length (512).
Number o

KeyboardInterrupt: 